# Raster resampling processing task

In this notebook, we will demonstrate resampling i.e. interpolating a raster dataset to a given different coordinate grid at a different resolution, with `scipy`. We will plot the raster layer using `folium`.

This is useful when you are doing a spatial join on 2 rasters which are defined on different grids - you will need to resample one raster onto the other's grid.

Datasets used:

- UK gridded 2000 population density data from [CIESIN](https://sedac.ciesin.columbia.edu/data/collection/gpw-v4), downloaded for the UK from [DIVA-GIS](http://www.diva-gis.org/gdata). Converted to geotiff using QGIS. Data resolution at roughly ~2.5km. The raster origin is at `lon,lat=-8.266667,60.93333` (top left, EPSG:4326) and the grid is 458x500.

## 0. Install Python packages

In [ ]:
!pip install numpy folium rasterio scipy

## 1. Load data

First connect Databricks to your datalake.

In [ ]:
datalake = "/dbfs/mnt/copgeospatial"

In [ ]:
import numpy as np
import rasterio

In [ ]:
dataset = rasterio.open(f"{datalake}/gbr_pop.tif")

In [ ]:
bounds = dataset.bounds
# BoundingBox(left=-8.266667, bottom=49.899995, right=1.8416677999999997, top=60.93333)

In [ ]:
dataset.shape
# (500, 458)

In [ ]:
population = dataset.read(1)

## 2. Process data

Get original grid coordinates

In [ ]:
orig_mgrid = np.meshgrid(*map(np.arange, dataset.shape))

orig_grid = np.vstack(tuple(map(
    np.ravel, 
    map(
        np.array, 
        rasterio.transform.xy(dataset.transform, *orig_mgrid)
    )
))).T

orig_grid.shape
# (229000, 2)

Define a completely new grid to resample onto, with resolution of thirty [seconds](https://en.wikipedia.org/wiki/Degree_(angle)#Subdivisions) (~1km):

In [ ]:
THIRTY_SECONDS = 0.0083333333333333

new_mgrid = np.mgrid[bounds.left:bounds.right:THIRTY_SECONDS, bounds.bottom:bounds.top:THIRTY_SECONDS]

new_grid = np.vstack(tuple(map(
    np.ravel,
    new_mgrid
))).T

new_grid.shape
# (1608550, 2)

Resample raster onto new grid using `scipy`

In [ ]:
from scipy.interpolate import griddata

In [ ]:
population_resampled = griddata(
    orig_grid, 
    np.where(population == -9999, np.nan, population).ravel(order='F'), # column-first ravel
    new_grid,
    method='linear'
)

In [ ]:
population_resampled = np.nan_to_num(
    population_resampled.reshape(*new_mgrid.shape[1:]), 
    nan=-9999
)

## 3. Visualise data on map

We use a custom colormap to deal with `nodata` values. We also must use `mercator_project=True` to reproject our raster into the map's underlying CRS.

In [ ]:
import folium
def colormap_func(maxi, nodata=-9999):
    return lambda x: (1,0,0,0 if x == nodata else max(0, x / maxi))

In [ ]:
m = folium.Map(location=[54.44, -3.5], zoom_start=12, tiles="CartoDB positron")

folium.raster_layers.ImageOverlay(
    population,
    ((bounds.bottom, bounds.left), (bounds.top, bounds.right)),
    colormap=colormap_func(population.max()),
    mercator_project=True
).add_to(m)

folium.LayerControl().add_to(m)
m.fit_bounds(m.get_bounds()) 

In [ ]:
m1 = folium.Map(location=[54.44, -3.5], zoom_start=12, tiles="CartoDB positron")

folium.raster_layers.ImageOverlay(
    np.flipud(population_resampled.T),
    ((bounds.bottom, bounds.left), (bounds.top, bounds.right)),
    colormap=colormap_func(population_resampled.max()),
    mercator_project=True
).add_to(m1)

folium.LayerControl().add_to(m1)
m1.fit_bounds(m1.get_bounds()) 

In [ ]:
m.save('/dbfs/mnt/copgeospatial/05_output.html')
m1.save('/dbfs/mnt/copgeospatial/05_output_1.html')

### Display map

Original raster of population on ~2.5km grid

In [3]:
from IPython.display import IFrame
IFrame("05_output.html", width="100%", height="500")

Resampled raster on ~1km grid

In [4]:
IFrame("05_output_1.html", width="100%", height="500")